In [ ]:
import numpy as np
import numpy.random as random

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import Concatenate

import data_generator

In [ ]:
class DeepIV():
    """Deep Instrumental Variables
    This class is an implemenatation of an approach using
    deep neural networks to estimate the causal effect of treatment T
    using a set of intrumental variables Z.
    
    See https://arxiv.org/abs/1612.09596
    """
    def __init__(self, num_instruments, num_features, num_treatments,
                 hidden_layer_sizes = [128, 64, 32], 
                 num_gaussians = 10, dropout_rate = 0.2,
                 l2_penalty = 0.0001, activation_function = "relu",
                 optimizer = "adam"):
        
        instruments = Input(shape = (num_instruments,), name = "instruments")
        features = Input(shape = (num_features,), name = "features")
        treatments = Input(shape = (num_treatments,), name = "treatment")
        
        instruments_and_features = Concatenate(axis=1)([instruments, features])
        features_and_treatments = Concatenate(axis=1)([features, treatments])
        
        estimated_treatments = architectures.feed_forward_net(
            instruments_and_features, 
            lambda x: densities.mixture_of_gaussian_output(x, num_gaussians),
            hidden_layers = hidden_layer_sizes, 
            dropout_rate = dropout_rate,
            l2 = l2_penalty, 
            activations = activation_function)
        
        treatment_model = Treatment(inputs=[instruments, features], outputs=estimated_treatments)
        treatment_model.compile(optimizer, loss="mixture_of_gaussians", n_components = num_gaussians)
        
        estimated_response = architectures.feed_forward_net(
            features_and_treatments, 
            Dense(1),
            hidden_layers = hidden_layer_sizes,
            dropout_rate=dropout_rate,
            l2 = l2_penalty,
            activations = activation_function)
        
        response_model = Response(
            treatment = treatment_model,
            inputs = [features, treatments],
            outputs = estimated_response)
        response_model.compile(optimizer, loss = "mse")
        
        self.treatment_model = treatment_model
        self.response_model = response_model
        
        
    def fit(self, Z, X, T, Y, epochs = 300, batch_size = 100, verbose = True):
        self.treatment_model.fit([Z, X], T, epochs=epochs, batch_size=batch_size, verbose=verbose)
        self.response_model.fit([Z, X], Y, epochs=epochs, batch_size=batch_size, 
                                samples_per_batch=2, verbose=verbose)
        

    def get_expected_representation(self, X, Z, n_samples=100):
        return self.response_model.expected_representation(X, Z, n_samples=n_samples)
    
    def get_eta_bar(self, X, Z):
        return self.get_expected_representation(X,Z)
    
    def get_conditional_representation(self, X, T):
        return self.response_model.conditional_representation(X, T)
    
    def get_eta(self, X, T):
        return self.get_conditional_representation(X,T)
    
    def predict(self, X, T):
        return self.response_model.predict([X, T])

In [ ]:
N = 5000
images = False
def datafunction(n, s, images=images, test=False):
    return data_generator.demand(n=n, seed=s, ypcor=0.5, use_images=images, test=test)

x, z, t, y, g_true = datafunction(N, 1)

num_instruments = z.shape[1]
num_features = x.shape[1]
num_treatments = t.shape[1]


In [ ]:
deepiv = DeepIV(num_instruments, num_features, num_treatments)
deepiv.fit(z,x,t,y, verbose=False, epochs=100)

In [ ]:
np.sum((deepiv.predict(x,t) - y)**2)

In [ ]:
deepiv.get_eta(x,t).shape

In [ ]:
deepiv.get_eta_bar(x,z).shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot()